In [4]:
import pandas as pd
import sqlalchemy
from database_connections import connect_to_postgres
import unidecode
import yaml
import json
from requests import get
from urllib import quote
import numpy as np
from bs4 import BeautifulSoup
import psycopg2

In [5]:
connection, cursor = connect_to_postgres()


Connected to server joshuacook.me.


In [6]:
cursor.close()
connection.close()

In [ ]:
connection.status
connection ,cursor = connect_to_postgres()
sql = """
    SELECT * FROM category;
    """
cursor.execute(sql)

print cursor.fetchone()
print cursor.fetchone()
cursor.close()
connection.close()

In [ ]:
connection.status
connection ,cursor = connect_to_postgres(location = 'remote')
sql = """
        INSERT INTO category (category_id, category_name)
        VALUES (757471,'Sandwiches');
    """
cursor.execute(sql)

connection.close()

In [ ]:
def create_query_param_string(params):
    param_list = [key+'='+value for key, value in params.items()]
    return '?'+'&'.join(param_list)
def wikipedia_page_format(page):
    return page.replace(' ','_')

In [ ]:
def parse_pages_from_json(response_json):
    return response_json['query']['pages']

def parse_headings_from_json(response_json):
    return response_json['mobileview']['sections']


In [ ]:
def wikipedia_get_cagegory(category):
    params = { 'action' : 'query',
               'format' : 'json',
               'prop' : 'extracts',
               'exlimit' : 'maxl'
             }
    params['titles'] = 'Category:'+quote(category)
    query_param_string = create_query_param_string(params)
    response = get(base_url+query_param_string)
    
    try:
        return parse_pages_from_json(response.json())
    except:
        return response

def wikipedia_get_pages_for_category(category):
    params = { 'action' : 'query',
               'format' : 'json',
               'list' : 'categorymembers',
               'cmlimit' : 'max'
             }
    
    params['cmtitle'] = 'Category:'+wikipedia_page_format(category)
    query_param_string = create_query_param_string(params)
    response = get(base_url+query_param_string)
    
    return response.json()['query']['categorymembers']
    
    try:
        return parse_headings_from_json(response.json())
    except:
        return response
    
def wikipedia_get_page(title, category=False):
    """ Get the text of a wikipedia page"""
    
    params = { 'action' : 'query',
               'format' : 'json',
               'prop' : 'extracts',
               'exlimit' : 'maxl'
        
    }
    
    if category:
        title = "Category:"+title
        
    params['titles'] = wikipedia_page_format(title)
    
    query_param_string = create_query_param_string(params)
    
    response = get(base_url+query_param_string)
    
    return response.json()['query']['pages']


In [ ]:
def add_category_to_database (category_id, category_name):
    connection ,cursor = connect_to_postgres(location = 'remote')
    insert_category_sql = """
    INSERT INTO category
    (category_id, category_name) VALUES ({}, {});
    """.format(category_id,category_name)
    cursor.execute(insert_category_sql)
    connection.commit()
    connection.close()

In [ ]:
base_url = "https://en.wikipedia.org/w/api.php"


In [ ]:
category_dict = wikipedia_get_cagegory('sandwiches')

In [ ]:
category_number = int(unidecode.unidecode(category_dict.keys()[0]))

In [ ]:
# insert category_number and 'sandwiches' in the database, "category" table.

In [ ]:
pages_list= wikipedia_get_pages_for_category('sandwiches')


In [ ]:
pages_list
page_id_to_query =[id['pageid'] for id in pages_list ]
page_name_to_query = [id['title'] for id in pages_list ]

In [ ]:
page_df = pd.DataFrame(zip (page_id_to_query, page_name_to_query),columns =['pageid','title'])

In [ ]:
page_df = page_df[np.invert(page_df['title'].str.contains('Category:', na = False))].head()

In [ ]:
page_df['query_string'] = page_df['title'].apply(wikipedia_page_format)

In [ ]:
page_df['html'] = page_df['title'].apply(wikipedia_get_page)

In [ ]:
def soup_text (wiki_stream):
    return BeautifulSoup(wiki_stream[wiki_stream.keys()[0]][u'extract'],'html.parser').get_text()

In [ ]:
page_df['page_text'] = page_df['html'].apply(soup_text)

In [ ]:
temp1 = wikipedia_get_page('Afghani_burger')
soup_text(temp1)

In [ ]:
page_df

In [ ]:
page_df_export = page_df[['pageid','title','page_text']]
             
page_df_export.columns = ['page_id','title','page_content']

In [ ]:
page_df_export['category_id'] = category_number

In [ ]:
engine.url


In [ ]:
page_df_export[['page_id', 'title']].to_sql('page',engine,if_exists='append', index = False)

In [ ]:
engine.dispose()

In [ ]:
category_number

In [ ]:
update_sql = """
    INSERT INTO category
    (category_id, category_name) VALUES (757471, 'Sandwiches');
    """
cursors.execute(update_sql)
connection.commit()
connection.close()

In [ ]:
import wikipedia
temp2 = wikipedia.page('Gua Bao')
print "pageid: {}".format(temp2.pageid)
print "text: " +temp2.content

In [ ]:
import psycopg2
from database_connections import connect_to_postgres, assign_connection_values

In [ ]:
cred = {'remote': {"user" : "chris", "password" : "till means newspaper wear",
                   'host' : "joshuacook.me", 'dbname' :'project_6'},
 'local' :  {"user_name" : "chris", "password" : "What44for",'host' : "localhost", 'database' : "project6"}
 }

connection = psycopg2.connect(**cred['remote'])

In [ ]:
def connect_to_postgres (location = 'remote', port = '5432'):

    with open('Database_credentials', 'r') as f:
        credentials =  yaml.load(f)
    print credentials[location]
    
    try:
        connection = psycopg2.connect(**credentials[location])
        print "Connected to server {}.".format(credentials[location]['host'])
        return connection, connection.cursor()
    except:
        print 'FAILED to connect to server {}.  Trying local server.'.format(credentials[location]['host'])
        try:
            connection = psycopg2.connect(**credentials['local'])
            print "Conencted to localhost."
            return connection, connection.cursor()
        except:
            print "No Database is available"
            pass 

In [ ]:
connect_to_postgres()

In [ ]:
connection.close()